In [2]:
from manim import *
config.set_width = "80%"

Manim Community v0.16.0

In [7]:
%%manim -qm -v WARNING Ex1
import numpy as np

class Ex1(Scene):
    def setup(self):
        equation = MathTex(r"f\left( x \right)=x^{2}").scale(2).to_corner(UR)
        self.add(equation)
        self.number_line1 = NumberLine(
            x_range=[0,10,1],
            include_numbers=True
        ).to_edge(LEFT).shift(UP)
        self.number_line2 = NumberLine(
            x_range=[0,100,10],
            length=12,
            include_numbers=True
        ).scale(1.05).to_edge(LEFT).shift(DOWN*2)
        self.add(self.number_line1, self.number_line2)
    def construct(self):
        from common.custom.custom_mobject import TickDecimalNumber
        decimal1 = TickDecimalNumber(
            number=0,
            tick_color=BLUE
        )
        decimal2 = TickDecimalNumber(
            number=0,
            tick_color=RED
        )
        value_tracker = ValueTracker(0)        
        def update_tick_decimal_factory(number_line, f):
            def func(obj):
                obj.get_decimal_number().set_value(f(value_tracker.get_value()))
                obj.move_to(number_line.n2p(f(value_tracker.get_value())) + np.array([-(obj.get_tick().get_center()[0] - obj.get_center()[0]),0,0]) + UP)
            return func
        
        decimal1.add_updater(update_tick_decimal_factory(self.number_line1, lambda x:x))
        decimal2.add_updater(update_tick_decimal_factory(self.number_line2, lambda x:x**2))
        
        self.add(value_tracker, decimal1, decimal2)
        self.play(value_tracker.animate.set_value(0))
        self.wait(1)
        move_to = [3,1,10]
        for i in move_to:
            self.play(value_tracker.animate.set_value(i), run_time=3)
            self.wait()
        self.wait()

In [8]:
%%manim -qm -v WARNING Ex2
import math

class Ex2(Scene):
    def setup(self):
        a = MathTex("a", "=")
        b = MathTex("b", "=")
        c = MathTex("c", "=")
        self.group = VGroup(a,b,c).arrange(DOWN, buff=1).shift(RIGHT*2+DOWN*2)
        colors = [RED, BLUE, YELLOW]
        for i, color in zip((a,b,c), colors):
            i[0].set_color(color=color)
        self.add(self.group)
        
        equation1 = MathTex(r"cos(at) + \frac{cos(bt)}{2}+\frac{sin(bt)}{3}").to_corner(UR)
        equation2 = MathTex(r"sin(at)+\frac{sin(bt)}{2}+\frac{cos(ct)}{3}").next_to(equation1, DOWN, aligned_edge=LEFT)
        group2 = VGroup(equation1, equation2)
        brace = Brace(group2, LEFT)
        c = MathTex("C").next_to(brace, LEFT)
        t = MathTex(r"t\in \left[ 0,2\pi \right]").next_to(group2, DOWN, aligned_edge=RIGHT)
        self.add(group2, c, brace, t)
        
    def construct(self):
        trackers = [ValueTracker() for _ in range(3)]
        trackers[0].set_value(1)
        print([t.get_value() for t in trackers])
        
        def func_factory(value):
            def func(obj):
                obj.set_value(value.get_value())
            return func
            
        decimals = [DecimalNumber(unit=r"^\circ").add_updater(func_factory(tracker)) for tracker in trackers]
        # decimals[0].add_updater(lambda x: x.set_value(trackers[0].get_value()))
        # decimals[1].add_updater(lambda x: x.set_value(trackers[1].get_value()))
        # decimals[2].add_updater(lambda x: x.set_value(trackers[2].get_value()))
        
        
        for i,j in zip(self.group, decimals):
            j.next_to(i, RIGHT)
        self.add(*trackers, *decimals)
        
        trackers[0].set_value(1)
        run_time=[1,10,10,10]
        target_value = [
            [1, 0, 0],
            [1, 15, 0],
            [15, 15, 0],
            [15, 15, 15]
           ]
        
        axes = Axes(
            x_range=[-2,2,1],
            y_range=[-2,2,1],
            x_length=4,
            y_length=4,
            axis_config={
                "include_numbers":False
            }
        ).shift(LEFT*3)

        def func(t):
            a,b,c = [i.get_value() for i in trackers]
            return [math.cos(a*t)+ math.cos(b*t)/2 + math.sin(c*t)/3,
                    math.sin(a*t)+ math.sin(b*t)/2 + math.cos(c*t)/3,
                    0]
            
        plot = axes.plot_parametric_curve(func, t_range=[0, 2*math.pi, 0.01], color=RED)
        
        def update_plot(obj):
            new_plot = axes.plot_parametric_curve(func, t_range=[0, 2*math.pi, 0.01], color=RED)
            plot.become(new_plot)
        
        plot.add_updater(update_plot)
        self.add(axes, plot)
        
        for target, time in zip(target_value, run_time):
            self.play(*[tracker.animate.set_value(value) for tracker, value in zip(trackers, target)],  run_time=time, rate_func=linear)
            self.wait(2)
        

[1.0, 0.0, 0.0]


In [9]:
%%manim -qm --disable_caching -v WARNING Ex3

class Ex3(Scene):
    def setup(self):
        self.axes = Axes(
            x_range=[-1,10,1],
            y_range=[-1,6,1],
            axis_config={
                "include_numbers": True
            }
        )
        func = lambda x: 0.1 * (x - 2) * (x - 8) * (x - 5) + 3
        self.plot = self.axes.plot(func, x_range=[0.8, 9, 0.05], color=RED)
        self.add(self.axes, self.plot)
        
    def construct(self):
        value = ValueTracker(2)
        dot = Dot(color=YELLOW)
        
        def update_dot(dot):
            dot.move_to(self.axes.i2gp(value.get_value(), self.plot))
        
        dot.add_updater(update_dot)
        
        def update_line(line):
            new_line = self.axes.get_lines_to_point(dot.get_center())
            lines.become(new_line)
        
        lines = VGroup().add_updater(update_line)
        
        def update_derivative(deri):
            current_value = value.get_value()
            p1 = self.axes.i2gp(current_value, self.plot)
            p2 = self.axes.i2gp(current_value+0.01, self.plot)
            new_deri = Line(LEFT, RIGHT, color=YELLOW).rotate(angle_of_vector(p1-p2)).move_to(p1)
            derivative.become(new_deri)
        
        derivative = Line().add_updater(update_derivative)
        
        self.add(value, dot, derivative, lines)
        self.play(value.animate.set_value(2), run_time=1)
        self.play(value.animate.set_value(8), run_time=5)

In [10]:
%%manim -qm -v WARNING Ex4

class Ex4(Scene):
    def setup(self):
        pass
    
    def construct(self):
        import numpy as np
        partical = 30
        
        a = np.array([-2,-2,0])
        b = np.array([-2,2,0])
        c = np.array([2,2,0])
        d = np.array([2,-2,0])
        dots = VGroup(*[Dot(i) for i in (a,b,c,d)])
        ab = Line(a, b)
        bc = Line(b, c)
        cd = Line(c, d)
        bezi = bezier([a,b,c,d])
        points = [bezi(i/partical) for i in range(partical+1)]
        curve = VMobject(color=RED).set_points_smoothly(points)
        self.play(*[Write(i) for i in (ab,bc,cd,curve, dots)])
        
        curve.generate_target()
        ab.generate_target()
        bc.generate_target()
        cd.generate_target()
        dots.generate_target()
        As = [
            (-4,-2,0),
            (-4,-2,0),
            (-4,2,0),
            (-4,2,0),
        ]
        Bs = [
            (-2,2,0),
            (-2,2,0),
            (-2,-2,0),
            (-2,-2,0),
        ]
        Cs = [
            (2,2,0),
            (2,2,0),
            (2,2,0),
            (2,2,0),
        ]
        Ds = [
            (2,-2,0),
            (4,-2,0),
            (4,-2,0),
            (1.5,-3.5,0),
        ]
        for a,b,c,d in zip(As, Bs, Cs, Ds):
            a = np.asarray(a)
            b = np.asarray(b)
            c = np.asarray(c)
            d = np.asarray(d)
            dots.target = VGroup(*[Dot(i) for i in (a,b,c,d)])
            ab.target = Line(a,b)
            bc.target = Line(b,c)
            cd.target = Line(c,d)
            
            bezi = bezier([a,b,c,d])
            points = [bezi(i/partical) for i in range(partical+1)]
            curve.target = VMobject(color=RED).set_points_smoothly(points)
            self.play(*[
                MoveToTarget(i) for i in [curve, ab, bc, cd, dots]
            ])
            self.wait()
        pass

In [11]:
%%manim -qm -v WARNING Ex5
from datetime import datetime

class Ex5(Scene):
    def setup(self):
        self.now = datetime.now()
        self.hour = self.now.hour + self.now.minute/60
        self.minute = self.now.minute
        print(self.hour)
        print(self.minute)
        self.circle = Circle().scale(2).rotate(PI/2).flip(UP)
        self.add(self.circle)
        self.list_min = [self.circle.point_from_proportion(i/60) for i in range(60)]
        self.list_line = [Line(i, self.circle.get_center()) for i in self.list_min]
        self.hour_hand = Arrow(self.circle.get_center(), self.list_line[0].point_from_proportion(0.4), buff=0).set_color(RED).rotate(-self.hour / 6 * PI, about_point=self.circle.get_center())
        self.minute_hand = Arrow(self.circle.get_center(), self.list_line[0].point_from_proportion(0.15), buff=0).set_color(BLUE).rotate(-self.minute / 30 * PI, about_point=self.circle.get_center())
        self.add(*[line.get_subcurve(0, 0.1) if i%5==0 else line.get_subcurve(0, 0.02) for i, line in enumerate(self.list_line)])
        self.add(self.hour_hand)
        self.add(self.minute_hand)
        self.add(*[Text("{hour:.0f}".format(hour=12 if i==0 else i/5)).scale(0.4).move_to(line.point_from_proportion(0.2)) for i, line in enumerate(self.list_line) if i%5 == 0])
        
    def construct(self):
        self.target_hour = 20
        self.target_minute = 25
        self.target_hour += self.target_minute / 60
        value = ValueTracker(self.hour*60)
        self.add(value)
        
        def update_hand_factory(arrow, f):
            def func(obj):
                new_hand = arrow.copy().rotate(f(value.get_value()), about_point=self.circle.get_center())
                obj.become(new_hand)
            return func
        
        self.hour_hand.add_updater(update_hand_factory(Arrow(self.circle.get_center(), self.list_line[0].point_from_proportion(0.4), buff=0).set_color(RED), lambda x: -(x/60)/6*PI ))
        self.minute_hand.add_updater(update_hand_factory(Arrow(self.circle.get_center(), self.list_line[0].point_from_proportion(0.15), buff=0).set_color(BLUE), lambda x: -(x%60)/30*PI ))
        
        self.play(value.animate.set_value(self.target_hour*60), run_time=10)
        self.wait()

17.566666666666666
34


In [12]:
%%manim -qm --disable_caching -v WARNING Ex6
from common.utils.color_utils import interpolate_color_range

class Ex6(Scene):
    def setup(self):
        self.axes = Axes(
            x_range=[-1, 10, 1],
            y_range=[-1, 6, 1],
            axis_config={
                "include_numbers": True
            }
        )
        func = lambda x: 0.1 * (x - 2) * (x - 8) * (x - 5) + 3
        self.plot = self.axes.plot(func, x_range=[0.8, 9], color=RED)
        self.add(self.axes, self.plot)
        
    def create_field(self, x_range):
        x = (x_range[1]-x_range[0])/x_range[2]
        
        rec_kwargs = {"stroke_width":0, "fill_opacity": 1}
        
        def get_color(i):
            return interpolate_color_range(GREEN, TEAL, i/x_range[2])
        
        def create_rectangle(i):
            start = i*x + x_range[0]
            end = (i+1)*x + x_range[0]
            dl = self.axes.c2p(start, 0)
            dr = self.axes.c2p(end, 0)
            ul = self.axes.i2gp(start, self.plot)
            ur = self.axes.i2gp(end, self.plot)
            return VMobject(fill_color=get_color(i), **rec_kwargs).set_points_as_corners([dl, dr, ur, ul,])
        
        return VGroup(*[create_rectangle(i) for i in range(x_range[2])])
        
    def construct(self):
        value_change=(4, 8, 6, 9)
        value = ValueTracker(value_change[0])
        self.add(value)
        
        def update_triangle(obj):
            x = self.axes.c2p(value.get_value(),0)
            obj.move_to(x+np.array([0,-0.1,0]))
        
        triangle = Triangle(fill_color=WHITE, stroke_width=0, fill_opacity=1).set(width=0.2).add_updater(update_triangle)
        field = self.create_field([2,4,int((4-2)*10)]).add_updater(lambda x: x.become(self.create_field([2, value.get_value(), int((value.get_value()-2)*10)])))
        self.add(triangle)
        self.add(field)
        
        for i in value_change:
            self.play(value.animate.set_value(i), run_time=3)
            self.wait()

In [13]:
%%manim -qm --disable_caching -v WARNING Ex7
from common.utils.utils import distance_between_two_points

class Ex7(Scene):
    def setup(self):
        self.axes = Axes(
            x_range = [-5,5,1],
            y_range = [0,5, 1],
            x_length = 10,
            y_length = 5,
            axis_config = {
                "include_ticks":True,
                "include_tip": False,
            }
        )
        self.add(self.axes)
        
    def create_updater(self, func):
        return func().add_updater(lambda x: x.become(func()))
    
    def create_plot(self):
        i = self.value.get_value()
        return self.axes.plot(lambda x: 0.25*x**2+1, x_range=[-i, i, 0.01])
    
    def create_dots(self):
        i = self.value.get_value()
        return VGroup(*[Dot(self.axes.i2gp(x, self.plot), color=RED) for x in [i, -i]])
    
    def create_lines(self):
        i = self.value.get_value()
        return VGroup(
            *[DashedLine(self.axes.i2gp(x, self.plot), self.axes.c2p(x,0)) for x in (-i, i)],
            *[DashedLine(self.axes.i2gp(x, self.plot), self.axes.c2p(0,2)) for x in (-i, i)],
        )
    
    def create_circles(self):
        i = self.value.get_value()
        return VGroup(
            *[Circle(radius = distance_between_two_points(self.axes.i2gp(i, self.plot), self.axes.c2p(i)))\
              .move_to(self.axes.i2gp(i, self.plot))\
              for i in (-i, i)
             ]
        )
    
    def construct(self):
        self.value = ValueTracker(0)
        
        self.plot = self.create_updater(self.create_plot)
        self.dots = self.create_updater(self.create_dots)
        self.lines = self.create_updater(self.create_lines)
        self.circles = self.create_updater(self.create_circles)
        
        self.add(self.plot, self.dots, self.lines, self.circles)
        self.wait()
        self.play(self.value.animate.set_value(3), run_time=3)
        self.wait()

In [15]:
%%manim -qm -v WARNING Ex8

class Ex8(Scene):
    def setup(self):
        pass
    
    def construct(self):
        polygon  = Triangle(start_angle=0*DEGREES).scale(2).flip(np.array([1,0,0]))
        for i in range(4,20):
            self.play(Transform(polygon, RegularPolygon(n=i, start_angle=0*DEGREES).scale(2).flip(np.array([1,0,0]))), rate_func=linear)

In [14]:
%%manim -qm --disable_caching -v WARNING E8

class E8(Scene):
    def construct(self):
        circle = Circle().scale(3)
        vt = ValueTracker(3)
        polygon = always_redraw(
            lambda: Polygon(*[
                circle.point_from_proportion(i/vt.get_value())
                for i in range(int(vt.get_value()+0.999))
            ])
        )
        self.add(polygon)
        self.wait(0.5)
        self.play(
            vt.animate.set_value(10),
            run_time=10,
            rate_func=linear,
        )
        self.wait()